In [58]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

- no. of parametres needed for CNN layer 1 with 32 units if there one channel and we are using square filter with width n length 5 will be
- **5x5x1x32 = 800**
- then 2nd layer of CNN with 64 unit then
- **5x5x1x32*64 = 51200**

In [4]:
(X_train,Y_train), (X_test, Y_test) =  keras.datasets.mnist.load_data()

In [59]:
X_train.shape

(60000, 28, 28)

In [60]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64

stride_conv1 = 1
stride_conv2 = 64

conv1_k = 5  # 5x5 filter
conv2_k = 5

max_pool1_k = 2
max_pool2_k = 2 # 2x2 maxpooling

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//max_pool1_k*max_pool2_k)*(input_height//max_pool1_k*max_pool2_k)*n_conv2

In [61]:
weights = {
    'wc1': tf.Variable(tf.random.normal([conv1_k,conv1_k, input_channels, n_conv1])), #wc1 : weigthConvolutionLayer1
    'wc2': tf.Variable(tf.random.normal([conv2_k,conv2_k, n_conv1, n_conv2])),
    'wh1' : tf.Variable(tf.random.normal([input_size_to_hidden,n_hidden])),
    'wo' : tf.Variable(tf.random.normal([n_hidden,n_out])),
}
                       
biases = { 
    
    'bc1' : tf.Variable(tf.random.normal([n_conv1])),
    'bc2' : tf.Variable(tf.random.normal([n_conv2])),
    'bh1' : tf.Variable(tf.random.normal([n_hidden])),
    'bo' : tf.Variable(tf.random.normal([n_out])),
}

In [62]:
def conv(x,wts, bias, strides=1):
    #apply filter on images
    out = tf.nn.conv2d(x, wts, padding='SAME', strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

In [63]:
def maxpooling(x, k = 2):
    return tf.nn.max_pool(x,padding = 'SAME',ksize =[1,k,k,1], strides=[1,k,k,1])

In [64]:
def cnn(x, wts, b):
    tf.reshape(x, shape = [-1,input_height, input_width, input_channels])
    # passing it through CLayer
    conv1 = conv(x,weights['wc1'], biases['bc1'])
    conv1_pool = maxpooling(conv1, max_pool1_k )
    conv2 = conv(x,weights['wc2'], biases['bc2'])
    conv2_pool = maxpooling(conv2, max_pool2_k )
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_output,weights['wh1']),baises['bh1'])
    
    hidden_output = tf.nn.relu(hidden_output_before_activation)
    
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    
    return output

In [65]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [70]:
x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases, keep_prob)

TypeError: cnn() takes 3 positional arguments but 4 were given

In [71]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y))

NameError: name 'pred' is not defined

In [72]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

NameError: name 'cost' is not defined

In [73]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [74]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

NameError: name 'mnist' is not defined

In [75]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels, keep_prob:1.0})
correct_preds.sum()

NameError: name 'pred' is not defined